In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

# visualization
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('CustomerExit_dataset/BankCustomers.csv')

In [3]:
data.head()

RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0       2       0.00              1          1               1   
1       1   83807.86              1          0               1   
2       8  159660.80              3          1               0   
3       1       0.00              2          0               0   
4       2  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57       1  
3         93826.63       0  
4         79084.10       0

In [4]:
data.describe()

RowNumber    CustomerId   CreditScore           Age        Tenure  \
count  10000.00000  1.000000e+04  10000.000000  10000.000000  10000.000000   
mean    5000.50000  1.569094e+07    650.528800     38.921800      5.012800   
std     2886.89568  7.193619e+04     96.653299     10.487806      2.892174   
min        1.00000  1.556570e+07    350.000000     18.000000      0.000000   
25%     2500.75000  1.562853e+07    584.000000     32.000000      3.000000   
50%     5000.50000  1.569074e+07    652.000000     37.000000      5.000000   
75%     7500.25000  1.575323e+07    718.000000     44.000000      7.000000   
max    10000.00000  1.581569e+07    850.000000     92.000000     10.000000   

             Balance  NumOfProducts    HasCrCard  IsActiveMember  \
count   10000.000000   10000.000000  10000.00000    10000.000000   
mean    76485.889288       1.530200      0.70550        0.515100   
std     62397.405202       0.581654      0.45584        0.499797   
min         0.000000       1.000000      0.00000        0.000000   
25%         0.000000       1.000000      0.00000        0.000000   
50%     97198.540000       1.000000      1.00000        1.000000   
75%    127644.240000       2.000000      1.00000        1.000000   
max    250898.090000       4.000000      1.00000        1.000000   

       EstimatedSalary        Exited  
count     10000.000000  10000.000000  
mean     100090.239881      0.203700  
std       57510.492818      0.402769  
min          11.580000      0.000000  
25%       51002.110000      0.000000  
50%      100193.915000      0.000000  
75%      149388.247500      0.000000  
max      199992.480000      1.000000

In [5]:
statistics_of_data = []
for col in data.columns:
  statistics_of_data.append((col,
                             data[col].nunique(),
                             data[col].isnull().sum()*100/data.shape[0],
                             data[col].value_counts(normalize=True, dropna=False).values[0] * 100, 
                             data[col].dtype
                             ))
stats_df = pd.DataFrame(statistics_of_data, columns=['Feature', 'Uniq_val', 'missing_val', 'val_biggest_cat', 'type'])

In [6]:
stats_df.sort_values('missing_val', ascending=False)

Feature  Uniq_val  missing_val  val_biggest_cat     type
0         RowNumber     10000          0.0             0.01    int64
1        CustomerId     10000          0.0             0.01    int64
2           Surname      2932          0.0             0.32   object
3       CreditScore       460          0.0             2.33    int64
4         Geography         3          0.0            50.14   object
5            Gender         2          0.0            54.57   object
6               Age        70          0.0             4.78    int64
7            Tenure        11          0.0            10.48    int64
8           Balance      6382          0.0            36.17  float64
9     NumOfProducts         4          0.0            50.84    int64
10        HasCrCard         2          0.0            70.55    int64
11   IsActiveMember         2          0.0            51.51    int64
12  EstimatedSalary      9999          0.0             0.02  float64
13           Exited         2          0.0            79.63    int64

In [7]:
def exploreFeatures(col):
  top_n=10
  top_n = top_n if data[col].nunique() > top_n else data[col].nunique()
  print(f"{col} has {data[col].nunique()} unique values and type: {data[col].dtype}.")
  print(data[col].value_counts(normalize=True, dropna=False).head())

In [8]:
exploreFeatures('Geography')


Geography has 3 unique values and type: object.
France     0.5014
Germany    0.2509
Spain      0.2477
Name: Geography, dtype: float64


In [9]:
exploreFeatures('Gender')

Gender has 2 unique values and type: object.
Male      0.5457
Female    0.4543
Name: Gender, dtype: float64


In [10]:

exploreFeatures('Tenure')

Tenure has 11 unique values and type: int64.
2    0.1048
1    0.1035
7    0.1028
8    0.1025
5    0.1012
Name: Tenure, dtype: float64


In [11]:
#Covert Gender and geography in dummy valiable/ ONE HOT ENCODING

states=pd.get_dummies(data['Geography'],drop_first=True)
gender=pd.get_dummies(data['Gender'],drop_first=True)

In [12]:
#concatenate the remaining dummies columns
data=pd.concat([data,states,gender],axis=1)

#drop the columns as it is no longer required

data=data.drop(['Geography','Gender'],axis=1)

In [13]:
X = data.drop(['Exited','RowNumber','CustomerId','Surname'],axis = 1)
y = data['Exited']



In [14]:
X.head()
y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

In [15]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [16]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [17]:
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [18]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [19]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(activation="relu", input_dim=11, units=6, kernel_initializer="uniform"))

# Adding the second hidden layer
classifier.add(Dense(activation="relu", units=6, kernel_initializer="uniform"))

# Adding the output layer
classifier.add(Dense(activation="sigmoid", units=1, kernel_initializer="uniform"))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [20]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 6)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, nb_epoch = 100)

c:\users\adeel\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/100
8000/8000 [==============================] - 2s 207us/step - loss: 0.4869 - accuracy: 0.7955
Epoch 2/100
8000/8000 [==============================] - 1s 89us/step - loss: 0.4284 - accuracy: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 89us/step - loss: 0.4242 - accuracy: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 96us/step - loss: 0.4199 - accuracy: 0.8151
Epoch 5/100
8000/8000 [==============================] - 1s 90us/step - loss: 0.4174 - accuracy: 0.8245
Epoch 6/100
8000/8000 [==============================] - 1s 89us/step - loss: 0.4154 - accuracy: 0.8259
Epoch 7/100
8000/8000 [==============================] - 1s 91us/step - loss: 0.4133 - accuracy: 0.8299
Epoch 8/100
8000/8000 [==============================] - 1s 90us/step - loss: 0.4122 - accuracy: 0.8316
Epoch 9/100
8000/8000 [==============================] - 1s 90us/step - loss: 0.4111 - accuracy: 0.8326
Epoch 10/100
8000/8000 [==============================] - 1s 90

8000/8000 [==============================] - 1s 92us/step - loss: 0.3997 - accuracy: 0.8354
Epoch 80/100
8000/8000 [==============================] - 1s 87us/step - loss: 0.3993 - accuracy: 0.8345
Epoch 81/100
8000/8000 [==============================] - 1s 88us/step - loss: 0.4000 - accuracy: 0.8356
Epoch 82/100
8000/8000 [==============================] - 1s 88us/step - loss: 0.3998 - accuracy: 0.8366
Epoch 83/100
8000/8000 [==============================] - 1s 94us/step - loss: 0.3998 - accuracy: 0.8364
Epoch 84/100
8000/8000 [==============================] - 1s 100us/step - loss: 0.3999 - accuracy: 0.8345
Epoch 85/100
8000/8000 [==============================] - 1s 95us/step - loss: 0.3995 - accuracy: 0.8331
Epoch 86/100
8000/8000 [==============================] - 1s 93us/step - loss: 0.3995 - accuracy: 0.8349
Epoch 87/100
8000/8000 [==============================] - 1s 85us/step - loss: 0.4000 - accuracy: 0.8334
Epoch 88/100
8000/8000 [==============================] - 1s 88us/s

In [22]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [24]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(y_test, y_pred)
accuracy=accuracy_score(y_test,y_pred)

In [26]:
cm

array([[1545,   50],
       [ 270,  135]], dtype=int64)

In [27]:
accuracy

0.84